In [2]:
# --- START OF FILE send_player_data_manual.py ---

import json
import time
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import KafkaError
import logging
import os
import signal

# --- Cấu hình Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# --- Cấu hình Kafka ---
# Sử dụng localhost:9092 nếu chạy từ host.
# Sử dụng kafka-h2dn:29092 nếu chạy từ 1 container trong mạng Docker.
KAFKA_BROKER_URL = os.getenv('KAFKA_BROKER_URL', 'localhost:9092')
# Topic để gửi dữ liệu cần dự đoán
KAFKA_TOPIC_PREDICT = 'soccer_to_predict'
REPLICATION_FACTOR = 1 # Vì chỉ có 1 broker Kafka trong compose
NUM_PARTITIONS = 1 # Thường là 1 cho topic này

# --- Xử lý tín hiệu ngắt ---
producer = None

def signal_handler(sig, frame):
    logger.info("Received interrupt signal. Cleaning up...")
    if producer:
        try:
            producer.flush(timeout=60)
            logger.info("Kafka producer flushed.")
        except Exception as flush_e:
            logger.error(f"ERROR during producer flush on signal: {flush_e}")
        try:
            producer.close()
            logger.info("Kafka producer closed.")
        except Exception as close_e:
             logger.error(f"ERROR during producer close on signal: {close_e}")
    logger.info("Cleanup completed. Exiting.")
    exit(0)

signal.signal(signal.SIGINT, signal_handler)
signal.signal(signal.SIGTERM, signal_handler)

# --- Kiểm tra và tạo Kafka topic ---
def check_and_create_topic():
    admin_client = None
    try:
        logger.info(f"Checking if topic '{KAFKA_TOPIC_PREDICT}' exists at {KAFKA_BROKER_URL}")
        admin_client = KafkaAdminClient(bootstrap_servers=[KAFKA_BROKER_URL])
        
        # List topics có thể gặp lỗi nếu Kafka chưa sẵn sàng ngay
        # Thử lại vài lần nếu cần thiết
        max_attempts = 5
        for attempt in range(max_attempts):
            try:
                topic_list = admin_client.list_topics()
                break # Success
            except KafkaError as e:
                if attempt < max_attempts - 1:
                    logger.warning(f"Failed to list topics, retrying in 5s (Attempt {attempt+1}/{max_attempts}): {e}")
                    time.sleep(5)
                else:
                    logger.error(f"Failed to list topics after {max_attempts} attempts: {e}")
                    raise # Re-raise the exception
        
        if KAFKA_TOPIC_PREDICT not in topic_list:
            logger.info(f"Topic '{KAFKA_TOPIC_PREDICT}' does not exist. Creating...")
            new_topic = NewTopic(
                name=KAFKA_TOPIC_PREDICT,
                num_partitions=NUM_PARTITIONS,
                replication_factor=REPLICATION_FACTOR
            )
            admin_client.create_topics(new_topics=[new_topic], validate_only=False)
            logger.info(f"Topic '{KAFKA_TOPIC_PREDICT}' created successfully.")
        else:
            logger.info(f"Topic '{KAFKA_TOPIC_PREDICT}' already exists.")
        
    except Exception as e:
        logger.error(f"Error checking/creating topic: {e}")
        # Không re-raise lỗi kết nối để producer vẫn có thể cố gắng gửi
        # raise # Uncomment để dừng nếu tạo topic thất bại
    finally:
        if admin_client:
            admin_client.close()


# Callback khi gửi thành công
def on_send_success(record_metadata):
    logger.info(f"Gửi thành công đến topic: {record_metadata.topic}, partition: {record_metadata.partition}, offset: {record_metadata.offset}")

# Callback khi gửi thất bại
def on_send_error(excp):
    logger.error(f"LỖI khi gửi tin nhắn: {excp}")

def get_player_data_from_input():
    """Lấy thông tin cầu thủ cần dự đoán từ input của người dùng."""
    print("\n" + "="*50)
    print(" Nhập thông tin cầu thủ mới để dự đoán giá trị thị trường ")
    print("="*50)

    data = {}
    try:
        # Các trường định danh và phân loại (strings)
        data['player_id'] = input("ID Cầu thủ (ví dụ: p123, duy trì duy nhất cho mỗi cầu thủ): ").strip()
        if not data['player_id']:
            logger.error("ID Cầu thủ không được để trống.")
            return None
        data['player_name'] = input("Tên cầu thủ: ").strip()
        data['club'] = input("CLB: ").strip()
        # Vị trí và chân thuận cần cho bước lọc trong script predict
        data['position'] = input("Vị trí (ví dụ: Midfielder, Forward, Defender. Goalkeeper sẽ bị bỏ qua): ").strip()
        data['strong_foot'] = input("Chân thuận (left, right, both): ").strip()

        # Các trường đặc trưng dùng để predict (cần là số/float)
        print("\n--- Nhập các chỉ số thống kê (Nhập số, sử dụng '.' cho thập phân): ---")
        
        try:
            data['age'] = float(input("Tuổi: "))
            data['appearances'] = float(input("Số lần ra sân: "))
            data['PPG'] = float(input("PPG (Points Per Game, ví dụ: 1.5): "))
            data['goals'] = float(input("Bàn thắng: "))
            data['assists'] = float(input("Kiến tạo: "))
            data['own_goals'] = float(input("Bàn phản lưới nhà: "))
            data['substitutions_on'] = float(input("Vào sân từ ghế dự bị: "))
            data['substitutions_off'] = float(input("Rời sân (bị thay ra): "))
            data['yellow_cards'] = float(input("Thẻ vàng: "))
            data['second_yellow_cards'] = float(input("Thẻ vàng thứ 2 (dẫn đến thẻ đỏ): "))
            data['red_cards'] = float(input("Thẻ đỏ trực tiếp: "))
            data['penalty_goals'] = float(input("Bàn thắng từ Penalty: "))
            data['minutes_per_goal'] = float(input("Số phút/bàn thắng (nhập 0 nếu chưa có bàn): "))
            data['minutes_played'] = float(input("Tổng số phút thi đấu: "))
            data['player_height'] = float(input("Chiều cao (m): "))
        except ValueError as ve:
            logger.error(f"Lỗi nhập liệu số: {ve}. Vui lòng nhập số hợp lệ.")
            return None

        # Thêm các trường khác nếu cần thiết hoặc để khớp schema đầy đủ (dù không dùng predict)
        # Bạn có thể hỏi hoặc gán giá trị mặc định
        data['nationality'] = input("Quốc tịch (để trống nếu không biết): ").strip() or "Unknown"
        # data['player_agent'] = input("Người đại diện (để trống nếu không biết): ").strip() or "None"
        data['contract_value_time'] = input("Hạn hợp đồng (năm hoặc text, để trống nếu không biết): ").strip() or "Unknown"
        # Các cột chỉ dùng cho thủ môn trong schema crawler, gán 0 cho non-GK
        data['goalkeeper_or_not'] = '0' # Gán mặc định 0 cho non-GK
        data['goals_conceded'] = 0.0
        data['clean_sheet'] = 0.0
        # market_value: Không nhập vì đây là giá trị cần dự đoán!
        data['market_value'] = None # Đặt là None hoặc 0, script dự đoán sẽ không dùng nó làm feature

        # Thêm timestamp để theo dõi thời gian nhập liệu
        data['ingest_timestamp_manual'] = int(time.time() * 1000) # Timestamp hiện tại theo epoch milliseconds
        data['source_type'] = 'manual_input' # Loại nguồn dữ liệu

        return data

    except Exception as e:
        logger.error(f"Lỗi không xác định khi lấy dữ liệu: {e}")
        return None

def main():
    global producer
    producer = None # Đảm bảo producer là None trước khi khởi tạo

    # Kiểm tra và tạo Kafka topic
    try:
        check_and_create_topic()
    except Exception as e:
        logger.error(f"Không thể kiểm tra hoặc tạo topic Kafka. Thoát chương trình. Lỗi: {e}")
        exit(1)


    # Khởi tạo Kafka producer
    try:
        logger.info(f"Connecting to Kafka Broker: {KAFKA_BROKER_URL}")
        producer = KafkaProducer(
            bootstrap_servers=[KAFKA_BROKER_URL],
            # Serialize dictionary sang JSON bytes
            value_serializer=lambda x: json.dumps(x, ensure_ascii=False).encode('utf-8'),
            # Tùy chọn cấu hình producer (tăng độ tin cậy)
            retries=5, # Thử lại 5 lần nếu gặp lỗi tạm thời
            acks='all' # Chờ tất cả các replicas xác nhận ghi
        )
        logger.info("Kết nối tới Kafka Producer thành công.")

    except Exception as e:
        logger.error(f"CRITICAL: Lỗi kết nối tới Kafka Producer: {e}")
        # Không thể kết nối Kafka là lỗi nghiêm trọng, thoát
        exit(1)


    try:
        while True:
            player_data = get_player_data_from_input()
            if player_data:
                logger.info(f"Chuẩn bị gửi dữ liệu cho cầu thủ ID: {player_data.get('player_id', 'N/A')}")
                try:
                    # Gửi dữ liệu đến Kafka topic bất đồng bộ
                    # Sử dụng add_callback và add_errback để log kết quả
                    future = producer.send(KAFKA_TOPIC_PREDICT, value=player_data)

                    # Có thể tùy chọn chờ kết quả gửi (blocking) nếu muốn đảm bảo ngay
                    # result = future.get(timeout=10)
                    # logger.info(f"Đã gửi dữ liệu cho cầu thủ '{player_data.get('player_name', 'N/A')}' (ID: {player_data.get('player_id', 'N/A')})")

                    # Hoặc dùng callback để xử lý kết quả gửi sau
                    future.add_callback(on_send_success)
                    future.add_errback(on_send_error)

                    # producer.flush() # Đảm bảo tất cả tin nhắn trong buffer được gửi đi ngay (tùy chọn)

                except Exception as e:
                    logger.error(f"Lỗi khi chuẩn bị hoặc gửi dữ liệu đến Kafka: {e}")
            else:
                 logger.warning("Dữ liệu cầu thủ không hợp lệ hoặc nhập liệu bị hủy.")

            
            another = input("\nBạn có muốn nhập thêm cầu thủ khác không? (yes/no): ").strip().lower()
            if another not in ['yes', 'y', 'có']: 
                break

    except Exception as e:
        logger.error(f"Lỗi trong vòng lặp nhập liệu chính: {e}", exc_info=True)
    finally:
        if producer:
            logger.info("Đảm bảo tất cả tin nhắn đang chờ được gửi...")
            try:
                producer.flush(timeout=60) # Chờ tối đa 60 giây để gửi hết buffer
                logger.info("Producer flush hoàn tất.")
            except Exception as flush_e:
                 logger.error(f"Lỗi khi flush producer: {flush_e}")
            logger.info("Đóng Kafka Producer.")
            producer.close()
        logger.info("Chương trình nhập liệu kết thúc.")

if __name__ == "__main__":
    main()

2025-05-27 07:44:25,712 - INFO - Checking if topic 'soccer_to_predict' exists at localhost:9092
2025-05-27 07:44:25,718 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2025-05-27 07:44:25,811 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=bootstrap-0 host=localhost:9092 <checking_api_versions_recv> [IPv6 ('::1', 9092, 0, 0)]>: Broker version identified as 2.6
2025-05-27 07:44:25,814 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2025-05-27 07:44:25,823 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, node_id=1 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2025-05-27 07:44:25,826 - INFO - <BrokerConnection client_id=kafka-python-2.1.5, 


 Nhập thông tin cầu thủ mới để dự đoán giá trị thị trường 

--- Nhập các chỉ số thống kê (Nhập số, sử dụng '.' cho thập phân): ---


2025-05-27 07:45:35,524 - INFO - Chuẩn bị gửi dữ liệu cho cầu thủ ID: 555555
2025-05-27 07:45:35,525 - INFO - <BrokerConnection client_id=kafka-python-producer-2, node_id=1 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2025-05-27 07:45:35,530 - INFO - <BrokerConnection client_id=kafka-python-producer-2, node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2025-05-27 07:45:35,532 - INFO - <BrokerConnection client_id=kafka-python-producer-2, node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. 
2025-05-27 07:45:35,620 - INFO - Gửi thành công đến topic: soccer_to_predict, partition: 0, offset: 5
2025-05-27 07:45:37,554 - INFO - Đảm bảo tất cả tin nhắn đang chờ được gửi...
2025-05-27 07:45:37,555 - INFO - Producer flush hoàn tất.
2025-05-27 07:45:37,556 - INFO - Đóng Kafka Producer.
2025-05-27 07:45:37,557 - INFO - <BrokerConnection 